# How does SuSiE perform given T=200 non-zero effects?

Assume using prior = 0.2. We investigate SuSiE performance given 200 non-zero effects.

## Results
**- Summary: SuSiE power is very low given 200 non-zero effects.** 

In [18]:
dscout.summary

effect_num,pve,mean_corX,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
200,0.01,0.0178845,0.0001,0.5000,1.0,1.000000,0.5000,1.0000000
200,0.02,0.0178845,0.0005,0.1667,1.0,1.000000,0.8333,1.0000000
200,0.03,0.0178845,0.0009,0.1000,1.1,1.000000,0.9000,0.9744434
200,0.05,0.0178845,0.0022,0.0833,1.0,1.000000,0.9167,1.0000000
200,0.10,0.0178845,0.0045,0.0217,1.0,1.243243,0.9783,1.0000000
200,0.20,0.0178845,0.0082,0.0000,1.0,1.708333,1.0000,1.0000000
200,0.40,0.0178845,0.0147,0.0000,1.0,2.940000,1.0000,1.0000000
200,0.50,0.0178845,0.0163,0.0000,1.0,3.260000,1.0000,1.0000000
200,0.70,0.0178845,0.0235,0.0000,1.0,4.700000,1.0000,1.0000000
200,0.90,0.0178845,0.0298,0.0033,1.0,5.980000,0.9967,1.0000000


## Code details

In [13]:
dscout_Q3 = readRDS('singlecell_Q3.rds')
dscout_Q3 = dscout_Q3[!is.na(dscout_Q3$sim_gaussian_large.output.file),]
dscout_Q3 = dscout_Q3[!is.na(dscout_Q3$susie_large.output.file),]

In [14]:
dscout_df = data.frame(dscout_Q3$sim_gaussian_large.effect_num, dscout_Q3$sim_gaussian_large.pve, 
                       dscout_Q3$score.hit, dscout_Q3$score.signal_num, dscout_Q3$score.cs_medianSize,
                       dscout_Q3$score.top_hit, dscout_Q3$sim_gaussian_large.mean_corX, dscout_Q3$susie_large.avg_purity)
names(dscout_df) = c('effect_num', 'pve','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [15]:
hitsum.summary = aggregate(hit ~ effect_num + pve, dscout_df, sum)
hitmean.summary = aggregate(hit ~ effect_num + pve, dscout_df, mean)
power.summary = hitmean.summary
power.summary$power = hitmean.summary$hit / power.summary$effect_num
fdr.summary = aggregate(cs_num ~ effect_num + pve, dscout_df, sum)
fdr.summary$fdr = round(1 - hitsum.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
setsize.summary = aggregate(cs_size ~ effect_num + pve, dscout_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve, dscout_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)
corX.summary = aggregate(corX ~ effect_num + pve, dscout_df, mean)
cs_num.summary = aggregate(cs_num ~ effect_num + pve, dscout_df, meannonzero)
dscout_df$avg_purity[is.na(dscout_df$avg_purity)]=0
purity.summary = aggregate(avg_purity ~ effect_num + pve, dscout_df, meannonzero)

In [16]:
dscout.summary = data.frame(power.summary$effect_num, power.summary$pve, corX.summary$corX,
                            power.summary$power, fdr.summary$fdr, setsize.summary$cs_size, 
                            cs_num.summary$cs_num, tophit.summary$tophit_rate, purity.summary$avg_purity)
names(dscout.summary) = c('effect_num', 'pve', 'mean_corX','power', 
                          'fdr', 'cs_size', 'cs_num','top_hit_rate', 'avg_purity')

In [17]:
is.nan.data.frame <- function(x)
do.call(cbind, lapply(x, is.nan))
dscout.summary[is.nan(dscout.summary)] = 0